In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

!pip install pymorphy2;
import pymorphy2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

import torch
!pip install transformers;
from transformers import AutoTokenizer, AutoModel

!pip install catboost;
import catboost

import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
set_stopwords = stopwords.words('russian')
analyzer = pymorphy2.MorphAnalyzer()

train_orig = pd.read_csv('../data/train.tsv', sep='\t')
test_orig = pd.read_csv('../data/test.tsv', sep = '\t')

In [3]:
train_orig.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


In [4]:
train = train_orig.copy()
test = test_orig.copy()

In [5]:
train.isnull().sum()

title      0
is_fake    0
dtype: int64

In [6]:
print(f'Размерность тренировочных данных: {train.shape}')
print(f'Размерность тестовых данных: {test.shape}')

Размерность тренировочных данных: (5758, 2)
Размерность тестовых данных: (1000, 2)


In [7]:
# классов у нас по ровну - дисбаланса нету)
train.is_fake.value_counts(normalize=True)

0    0.5
1    0.5
Name: is_fake, dtype: float64

In [8]:
print('Количество слов до удаления символов и стоп слов:')
print(train.title.str.split(' ').apply(lambda x: len(x)).sum())

print('Количество слов после удаления символов и стоп слов:')
# привожу к нижнему регистру, удаляю символы и стоп-слова
x_filtered = train.title.str.lower().str.split(' ').apply(lambda x: [i for i in x if (i.isalpha() & ~(i in set_stopwords))])
print(x_filtered.apply(lambda x: len(x)).sum())

Количество слов до удаления символов и стоп слов:
50237
Количество слов после удаления символов и стоп слов:
37617


In [9]:
# лемматизирую с помощью библиотеки pymorphy
x_lemmatized = x_filtered.apply(lambda x: [(analyzer.parse(word)[0]).normal_form for word in x ])

print(x_lemmatized.head())

0    [москвич, владимир, клутина, прийти, счёт, вме...
1    [агент, кокорин, назвать, езда, встречка, жите...
2    [госдума, рассмотреть, возможность, введение, ...
3    [фас, заблокировать, поставка, скоростной, тра...
4    [против, навальный, завести, дело, недоносител...
Name: title, dtype: object


In [10]:
print(f"Средняя длина настоящего сообщения: {round(x_lemmatized[train.is_fake==0].apply(lambda x: len(x)).mean(), 2)}")
print(f"Средняя длина фейкового сообщения: {round(x_lemmatized[train.is_fake==1].apply(lambda x: len(x)).mean(), 2)}")

Средняя длина настоящего сообщения: 5.81
Средняя длина фейкового сообщения: 7.26


In [11]:
top_fake_counter = Counter()
top_true_counter = Counter()

for row in x_lemmatized[train.is_fake==1]:
    top_fake_counter.update(row)
for row in x_lemmatized[train.is_fake==0]:
    top_true_counter.update(row)

In [12]:
print(top_fake_counter.most_common(5))
print(top_true_counter.most_common(5))

[('россия', 266), ('российский', 141), ('год', 133), ('запретить', 120), ('навальный', 115)]
[('россия', 230), ('российский', 143), ('новый', 118), ('год', 96), ('сша', 88)]


Наиболее встречаемые слова в каждом классе приблизительно одни и те же, но заметно, что средние длины фейков и настоящих сообщений отличаются.
Можно было бы статистически проверить гипотезу, что между классами средняя длина сообщения различается и в случае статистически значимых различий вручную добавить в качестве фичи длину сообщения, но я решил не усложнять 

In [13]:
x_final = x_lemmatized.str.join(' ')
x_final.head()

0    москвич владимир клутина прийти счёт вмешатель...
1    агент кокорин назвать езда встречка житейский ...
2    госдума рассмотреть возможность введение секре...
3    фас заблокировать поставка скоростной трамвай ...
4    против навальный завести дело недоносительство...
Name: title, dtype: object

#### 1 попытка - tfidf + random_forest

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_final, train.is_fake, stratify=train.is_fake, test_size = 0.2, random_state = 0)
tfidf = TfidfVectorizer(lowercase=False)
x_train, x_val = tfidf.fit_transform(x_train), tfidf.transform(x_val)

In [ ]:
forest = RandomForestClassifier(random_state=0)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_val)

In [ ]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.93      0.81       576
           1       0.90      0.62      0.73       576

    accuracy                           0.78      1152
   macro avg       0.80      0.78      0.77      1152
weighted avg       0.80      0.78      0.77      1152



#### 2 попытка - tfidf + фича длины текста + random_forest

In [ ]:
x_final_2 = pd.concat((x_final,
           x_final.apply(lambda x: len(x))
          ), axis=1
         )
x_final_2.columns = ['text', 'num_words']

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_final_2, train.is_fake, stratify=train.is_fake, test_size = 0.2, random_state = 0)
tfidf = TfidfVectorizer(lowercase=False)

# кодирую фичи через tfidf и сразу же конкатенирую с фичой "количество слов"
x_train = np.concatenate((tfidf.fit_transform(x_train['text']).toarray(),
                          x_train.num_words.values[:, np.newaxis]),
                         axis=1)

x_val = np.concatenate((tfidf.transform(x_val['text']).toarray(),
                        x_val.num_words.values[:, np.newaxis]),
                       axis=1)

In [ ]:
forest2 = RandomForestClassifier(random_state=0)
forest2.fit(x_train, y_train)
y_pred = forest2.predict(x_val)

In [ ]:
# Всё-таки эта фича оказалась полезной )
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.89      0.82       576
           1       0.87      0.70      0.78       576

    accuracy                           0.80      1152
   macro avg       0.81      0.80      0.80      1152
weighted avg       0.81      0.80      0.80      1152



#### 3 попытка - попробую использовать дистилрованную(уменьшенную) русскоязычную версию Bert

In [53]:
# на colab не работает git lfs
# на stackoverflow нашел решение проблемы в этом коде
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 94 not upgraded.
Need to get 6,800 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.1.2 [6,800 kB]
Fetched 6,800 kB in 1s (11.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [55]:
# скачал дистилрованную версию русскоязычного bert, который работает в разы быстрее
!git lfs clone  https://huggingface.co/cointegrated/rubert-tiny

          with new flags from `git clone`

`git clone` has been updated in upstream Git to have comparable
speeds to `git lfs clone`.
Cloning into 'rubert-tiny'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 79 (delta 37), reused 0 (delta 0)
Unpacking objects: 100% (79/79), done.


In [56]:
tokenizer = AutoTokenizer.from_pretrained("rubert-tiny")
model = AutoModel.from_pretrained("rubert-tiny")

Some weights of the model checkpoint at rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
# функция для получения эмбеддингов
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

(312,)


In [58]:
# получил эмбеддинги и сконкатенировал с фичой длины текста
x_final_bert = [embed_bert_cls(sentence, model, tokenizer) for sentence in x_final]
x_final_bert = np.vstack(x_final_bert)
x_final_bert_end = np.hstack((x_final_bert,
                              train.title.apply(lambda x: len(x)).values[:, np.newaxis]))

In [61]:
x_train, x_val, y_train, y_val = train_test_split(x_final_bert_end, train.is_fake, random_state = 0, stratify = train.is_fake, test_size = 0.2)

In [62]:
forest3 = RandomForestClassifier(random_state=0)
forest3.fit(x_train, y_train)
y_pred = forest3.predict(x_val)

In [63]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.82      0.80       576
           1       0.81      0.76      0.79       576

    accuracy                           0.79      1152
   macro avg       0.79      0.79      0.79      1152
weighted avg       0.79      0.79      0.79      1152



In [66]:
logreg = LogisticRegression(max_iter = 1000, random_state = 0)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_val)

In [67]:
print(classification_report(y_val, y_pred_logreg))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82       576
           1       0.83      0.79      0.81       576

    accuracy                           0.81      1152
   macro avg       0.82      0.81      0.81      1152
weighted avg       0.82      0.81      0.81      1152



**Вывод:** результат 3 итерации в случае леса хуже, чем на второй, хотя используется более сложная модель для получения эмбеддингов. Однако логистическая регрессия показала себя немного лучше, чем лес.

#### 4 попытка - препроцессинг как во 2 итерации, но использование catboost

In [ ]:
# параметр verbose - для контроля количества вывода информации
# catboost выводит излишне много текста во время обучения
boosting = catboost.CatBoostClassifier(verbose=100, random_state=0)

In [ ]:
x_final_2 = pd.concat((x_final,
           x_final.apply(lambda x: len(x))
          ), axis=1
         )
x_final_2.columns = ['text', 'num_words']

x_train, x_val, y_train, y_val = train_test_split(x_final_2, train.is_fake, stratify=train.is_fake, test_size = 0.2, random_state = 0)
tfidf = TfidfVectorizer(lowercase=False)

# кодирую фичи через tfidf и сразу же конкатенирую с фичой "количество слов"
x_train = np.concatenate((tfidf.fit_transform(x_train['text']).toarray(),
                          x_train.num_words.values[:, np.newaxis]),
                         axis=1)

x_val = np.concatenate((tfidf.transform(x_val['text']).toarray(),
                        x_val.num_words.values[:, np.newaxis]),
                       axis=1)

In [ ]:
boosting.fit(x_train, y_train)
y_pred = boosting.predict(x_val)
print(classification_report(y_val, y_pred))

Learning rate set to 0.019778
0:	learn: 0.6888867	total: 235ms	remaining: 3m 54s
100:	learn: 0.5373584	total: 7.1s	remaining: 1m 3s
200:	learn: 0.5048742	total: 14.2s	remaining: 56.6s
300:	learn: 0.4827345	total: 21.4s	remaining: 49.6s
400:	learn: 0.4656508	total: 28.5s	remaining: 42.6s
500:	learn: 0.4459051	total: 35.7s	remaining: 35.5s
600:	learn: 0.4279792	total: 42.7s	remaining: 28.4s
700:	learn: 0.4117548	total: 49.8s	remaining: 21.3s
800:	learn: 0.3974354	total: 56.7s	remaining: 14.1s
900:	learn: 0.3853614	total: 1m 3s	remaining: 6.99s
999:	learn: 0.3747131	total: 1m 10s	remaining: 0us
              precision    recall  f1-score   support

           0       0.76      0.86      0.81       576
           1       0.84      0.73      0.78       576

    accuracy                           0.79      1152
   macro avg       0.80      0.79      0.79      1152
weighted avg       0.80      0.79      0.79      1152



Ничего не дало. Используем 2 итерацию, но еще проведём дополнительный GridSearch для выявления лучших параметров

#### 5 попытка - использовать более тяжелый Bert

In [77]:
!git lfs clone https://huggingface.co/DeepPavlov/rubert-base-cased-sentence

          with new flags from `git clone`

`git clone` has been updated in upstream Git to have comparable
speeds to `git lfs clone`.
Cloning into 'rubert-base-cased-sentence'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 33 (delta 12), reused 0 (delta 0)
Unpacking objects: 100% (33/33), done.


In [14]:
tokenizer = AutoTokenizer.from_pretrained("./rubert-base-cased-sentence/")
model = AutoModel.from_pretrained("./rubert-base-cased-sentence/")
# model.cuda()  # раскоментируй, если у тебя есть GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(768,)


In [88]:
# получение эмбеддингов и конкатенация с фичей длины текста
x_final_bert = [embed_bert_cls(sentence, model, tokenizer) for sentence in x_final]
x_final_bert = np.vstack(x_final_bert)
x_final_bert_end = np.hstack((x_final_bert,
                              train.title.apply(lambda x: len(x)).values[:, np.newaxis]))

In [89]:
x_train, x_val, y_train, y_val = train_test_split(x_final_bert_end, train.is_fake, random_state = 0, stratify = train.is_fake, test_size = 0.2)

In [90]:
logreg = LogisticRegression(max_iter = 1000, random_state = 0)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_val)

In [91]:
print(classification_report(y_val, y_pred_logreg))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       576
           1       0.85      0.81      0.83       576

    accuracy                           0.84      1152
   macro avg       0.84      0.84      0.84      1152
weighted avg       0.84      0.84      0.84      1152



лучший результат! Моё решение - использовать полную модель rubert вместе с logistic regression. Осталось только подобрать наилучшие параметры для логистической регрессии

In [98]:
# словарь для поиска по сетке гиперпараметров
grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000], 'max_iter':[1000, 3000]}
logreg = LogisticRegression()
gridsearch_logreg = GridSearchCV(logreg, grid_values,scoring='f1')

In [99]:
gridsearch_logreg.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [1000, 3000], 'penalty': ['l1', 'l2']},
             scoring='f1')

In [101]:
gridsearch_logreg.best_score_

0.8475209954245283

In [102]:
gridsearch_logreg.best_params_

{'C': 10, 'max_iter': 1000, 'penalty': 'l2'}

In [105]:
# использую найденные лучшие параметры для логистической регрессии
logreg = LogisticRegression(max_iter = 1000, C=10,random_state = 0)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_val)

print(classification_report(y_val, y_pred_logreg))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87       576
           1       0.88      0.85      0.87       576

    accuracy                           0.87      1152
   macro avg       0.87      0.87      0.87      1152
weighted avg       0.87      0.87      0.87      1152



Финальный вариант модели:

1) Очистка текста от символов, приведение к нижнему регистру, лемматизация

2) Генерация эмбеддингов на основе модели RuBert + дополнительная фича длина символов 

3) Подаём в логистическую регрессию с гиперапараметрами: максимум_итераций = 1000, регуляризация = 10, порог = 0.5


## Обработка тестового датасета

In [15]:
test.head()

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0


In [16]:
def preprocess_df(df, stopwords, lemmatizer):
    '''
    df: датафрейм с тренировочными данными без столбца таргетов
    stopwords: список стоп-слов
    lemmatizer: лемматизатор от pymorphy2

    функция проводит предобработку сырого текста
    '''
    
    # приведение к нижнему регистру и удаление стоп-слов
    x_filtered = df.title.str.lower().str.split(' ').apply(lambda x: [i for i in x if (i.isalpha() & ~(i in stopwords))])
    
    # лемматизация
    x_lemmatized = x_filtered.apply(lambda x: [(lemmatizer.parse(word)[0]).normal_form for word in x ])

    return x_lemmatized

In [17]:
prep_text_test = preprocess_df(test, set_stopwords, analyzer).str.join(' ')
num_words_test = prep_text_test.apply(lambda x: len(x)).values[:, np.newaxis]

In [18]:
embeddings_test = [embed_bert_cls(sentence, model, tokenizer) for sentence in prep_text_test]

In [19]:
x_test = np.hstack((embeddings_test,
                    num_words_test))
x_test.shape

(1000, 769)

In [26]:
test_pred = logreg.predict(x_test)

In [33]:
submission = test_orig.copy()
submission['is_fake'] = test_pred

In [41]:
submission.to_csv('../predictions.tsv', sep='\t', index=False)